<a href="https://colab.research.google.com/github/ayyucedemirbas/machine_learning_algorithms/blob/master/time_series_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Univariate LSTM Models

Vanillia LSTM

In [1]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [2]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


In [3]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [4]:
model = Sequential([
                    LSTM(50, activation='relu', input_shape=(n_steps, n_features)),
                    Dense(1)
                    ])

In [5]:
model.compile(optimizer='adam', loss='mse')

In [6]:
model.fit(X, y, epochs=200, verbose=0)

In [7]:
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[102.38738]]


Stacked LSTM

In [8]:
modelstacked = Sequential([
                           LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)),
                           LSTM(50, activation='relu'),
                           Dense(1)])

In [9]:
modelstacked.compile(optimizer='adam', loss='mse')

In [10]:
modelstacked.fit(X, y, epochs=200, verbose=0)

In [11]:
yhat = modelstacked.predict(x_input, verbose=0)
print(yhat)

[[104.31474]]


Bidirectional LSTM

In [12]:
from keras.layers import Bidirectional

In [13]:
modelbidirectional = Sequential([
                                 Bidirectional(LSTM(50, activation='relu'), input_shape=(n_steps, n_features)),
                                 Dense(1)])

In [14]:
modelbidirectional.compile(optimizer='adam', loss='mse')

In [15]:
modelbidirectional.fit(X, y, epochs=200, verbose=0)

In [16]:
yhat = modelbidirectional.predict(x_input, verbose=0)
print(yhat)

[[101.49253]]


CNN LSTM

In [17]:
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
n_steps = 4
X, y = split_sequence(raw_seq, n_steps)
n_features = 1
n_seq = 2
n_steps = 2
X = X.reshape((X.shape[0], n_seq, n_steps, n_features))

In [18]:
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [19]:
modelcnn = Sequential([
                    TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, n_steps, n_features)),
                    TimeDistributed(MaxPooling1D(pool_size=2)),
                    TimeDistributed(Flatten()),
                    LSTM(50, activation='relu'),
                    Dense(1)])

In [20]:
modelcnn.compile(optimizer='adam', loss='mse')

In [21]:
modelcnn.fit(X, y, epochs=500, verbose=0)

In [22]:
x_input = array([60, 70, 80, 90])
x_input = x_input.reshape((1, n_seq, n_steps, n_features))
yhat = modelcnn.predict(x_input, verbose=0)
print(yhat)

[[100.80335]]


ConvLSTM

In [23]:
n_steps = 4
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, rows, columns, features]
n_features = 1
n_seq = 2
n_steps = 2
X = X.reshape((X.shape[0], n_seq, 1, n_steps, n_features))

In [24]:
from keras.layers import ConvLSTM2D

In [25]:
modelconv = Sequential([
                        ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)),
                        Flatten(),
                        Dense(1)])

In [26]:
modelconv.compile(optimizer='adam', loss='mse')

In [27]:
modelconv.fit(X, y, epochs=500, verbose=0)

In [28]:
x_input = array([60, 70, 80, 90])
x_input = x_input.reshape((1, n_seq, 1, n_steps, n_features))
yhat = modelconv.predict(x_input, verbose=0)
print(yhat)

[[103.395584]]


#Multivariate LSTM Models

Multiple Input Series

In [29]:
from numpy import hstack

In [30]:
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# define model
modelmis = Sequential([
                    LSTM(50, activation='relu', input_shape=(n_steps, n_features)),
                    Dense(1)])
modelmis.compile(optimizer='adam', loss='mse')
# fit model
modelmis.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = modelmis.predict(x_input, verbose=0)
print(yhat)

[[205.92073]]


Multiple Parallel Series

In [42]:
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# define model
modelmps = Sequential([
                       LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)),
                       LSTM(100, activation='relu'),
                       Dense(n_features)
])

modelmps.compile(optimizer='adam', loss='mse')
# fit model
modelmps.fit(X, y, epochs=400, verbose=0)
# demonstrate prediction
x_input = array([[70,75,145], [80,85,165], [90,95,185]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = modelmps.predict(x_input, verbose=0)
print(yhat)

[[100.07089  105.277824 205.52837 ]]


#Multi-Step LSTM Models

Vector Output Model

In [43]:
def split_sequence(sequence, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# split into samples
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
modelvom = Sequential([
                       LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)),
                       LSTM(100, activation='relu'),
                       Dense(n_steps_out)])

modelvom.compile(optimizer='adam', loss='mse')
# fit model
modelvom.fit(X, y, epochs=50, verbose=0)
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = modelvom.predict(x_input, verbose=0)
print(yhat)

[[119.02525 135.13393]]


Encoder-Decoder Model

In [44]:
from keras.layers import TimeDistributed

In [47]:
from keras.layers import RepeatVector

In [45]:
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# split into samples
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
y = y.reshape((y.shape[0], y.shape[1], n_features))

In [48]:
modeledm = Sequential([
                       LSTM(100, activation='relu', input_shape=(n_steps_in, n_features)),
                       RepeatVector(n_steps_out),
                       LSTM(100, activation='relu', return_sequences=True),
                       TimeDistributed(Dense(1))])

In [49]:
modeledm.compile(optimizer='adam', loss='mse')

In [50]:
modeledm.fit(X, y, epochs=100, verbose=0)

In [51]:
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = modeledm.predict(x_input, verbose=0)
print(yhat)

[[[103.50253]
  [117.00033]]]


#Multivariate Multi-Step LSTM Models

Multiple Input Multi-Step Output.

In [52]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out-1
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# covert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# define model
modelmimso = Sequential([
                         LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)),
                         LSTM(100, activation='relu'),
                         Dense(n_steps_out)])

modelmimso.compile(optimizer='adam', loss='mse')
# fit model
modelmimso.fit(X, y, epochs=200, verbose=0)
# demonstrate prediction
x_input = array([[70, 75], [80, 85], [90, 95]])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = modelmimso.predict(x_input, verbose=0)
print(yhat)

[[185.45682 206.75032]]


Multiple Parallel Input and Multi-Step Output

In [53]:
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# covert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
# define model
modelmpimso = Sequential([
                          LSTM(200, activation='relu', input_shape=(n_steps_in, n_features)),
                          RepeatVector(n_steps_out),
                          LSTM(200, activation='relu', return_sequences=True),
                          TimeDistributed(Dense(n_features))])

modelmpimso.compile(optimizer='adam', loss='mse')
# fit model
modelmpimso.fit(X, y, epochs=300, verbose=0)
# demonstrate prediction
x_input = array([[60, 65, 125], [70, 75, 145], [80, 85, 165]])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = modelmpimso.predict(x_input, verbose=0)
print(yhat)

[[[ 89.60106  95.45717 185.08041]
  [100.45827 105.4246  206.02652]]]
